In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

: 

Loading Data Sets

In [ ]:
df_train = pd.read_csv("/home/moamen/data_sets/1 project-nyc-taxi-trip-duration/split/train.csv")
df_val = pd.read_csv("/home/moamen/data_sets/1 project-nyc-taxi-trip-duration/split/val.csv")
df_test = pd.read_csv("/home/moamen/data_sets/1 project-nyc-taxi-trip-duration/split/test.csv")

How Many Rows and Features in Our Data Sets

In [ ]:
df_train.shape

In [ ]:
df_train.head()

In [ ]:
df_train.info()

We Have 7 numerical columns and we will add another one later and 3 object columns we will drop 1 of them (id)

Let's Check Is If The range Of :
1. Longitude [-90 , 90]
2. Latitude [-180, 180]
3. Passenger Numbers must be >= 0

In [ ]:
df_train.describe().T

The Columns' -We Have Mentioned- values Range is Natural

There Is Outlier or missleading rows in our(trip_duration) column, the third quartile is **1074 Sec** and the max is **2.2 * 10^6 sec** About 25 Day 
This is Incridble for a nyc taxi trip, we Need To Filter this Outliers, We Can Filter The Trips Exceedes One Day

In [ ]:
df_train = df_train[(df_train['trip_duration'] > 60) & (df_train['trip_duration'] < 86400)]  

We Will Extract a new Feature From These Features -> **Great Circle Distance**

In [ ]:
longitude_a = df_train['pickup_longitude'].to_numpy()
latitude_a = df_train['pickup_latitude'].to_numpy()
longitude_b = df_train['dropoff_longitude'].to_numpy()
latitude_b = df_train['dropoff_latitude'].to_numpy()

This is The Formula We Used To Generate The Great Circle Distance called **Haversine** Between Two Points If You Have 
Thier logitude and latitude

In [ ]:
def haversine(lat1: np.ndarray, lon1: np.ndarray, lat2: np.ndarray, lon2: np.ndarray) -> np.ndarray:
    R = 6371.0  # Radius of Earth in kilometers

    # Convert degrees to radians
    lat1 = np.radians(lat1)
    lon1 = np.radians(lon1)
    lat2 = np.radians(lat2)
    lon2 = np.radians(lon2)
    
    # Differences between coordinates
    dlat = lat2 - lat1
    dlon = lon2 - lon1

    # Haversine formula
    a = np.sin(dlat / 2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2.0)**2
    
    # Fix precision errors: ensure a is within [0, 1]
    a = np.clip(a, 0, 1)

    # Central angle
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))

    # Distance in kilometers
    distance = R * c
    return distance


In [ ]:
df_train.insert(loc=len(df_train.columns) -1, column='Distance', value=pd.DataFrame(haversine(longitude_a, latitude_a, longitude_b, latitude_b)))

Let's See The New Column Statistics

In [ ]:
df_train['Distance'].describe().T

In [ ]:
plt.figure(figsize=(10, 6))
plt.scatter(df_train['Distance'], df_train['trip_duration'], alpha=0.25)
plt.title("Using scatter")
plt.xlabel("Great Circle Distance")
plt.ylabel("Trip Duration")
plt.show()

We See The RealationShip **Is not Linear** because there a lot of Factors Affecting The Trip Duration
Ex: 
- The Distance Might Be Low But The road may have high traffic in this trip

Let's check how many null-values in the train, val and test set

In [ ]:
df_train.isna().sum()

**There isn't any null-values in the Original Data**, But We Have a few null values in the column we generated

**How not null data generates null data?!**

Let's Explore Is there **Invalid Data** in the origianl data set

In [ ]:
df_train.describe().T

For Now We Will Drop The NUll values in Distance, until we Make The BaseLine Model 

In [ ]:
df_train = df_train[df_train['Distance'].notnull()]
df_train.isna().sum()

In [ ]:
df_train['store_and_fwd_flag']

This flag indicates whether the trip record was held in vehicle memory before
sending to the vendor, aka “store and forward,” because the vehicle did not
have a connection to the server.
Y = store and forward trip
N = not a store and forward trip

Let's Viualize How Many Store and Forward Trips 

In [ ]:
df_train['store_and_fwd_flag'].value_counts()

In [ ]:
status = ['Y = store and forward trip', 'N = not a store and forward trip']
counts = [(df_train['store_and_fwd_flag'] == 'Y').sum(), (df_train['store_and_fwd_flag'] == 'N').sum()]
plt.bar(status, counts)
plt.title('store_and_fwd_status')
plt.xlabel('Status')
plt.ylabel('Counts')
plt.tight_layout()
plt.show()

Observation:
- Y is not zero but it is very small if we compare it with N

I Noticed The Time Of The Trip Maybe Affecting The Trip Duration, Imagin You Drive Your Car About 3 or 5 am
There is a big Chance To Reach Your Distnation Faster Than if You Drive Your Car in The Crowded Steets About 12 to 2 pm
,so We Can Categorize The Time Zones Into about 3 or 4 categoreis 
- 12 am to 4 am
- 5 am to 11 am
- 12 pm to 4 pm
- 5 pm to 11 pm

Let's Binning the Time Data

In [ ]:
df_train['pickup_datetime'] = pd.to_datetime(df_train['pickup_datetime'])

df_train.insert(loc=len(df_train.columns) - 1, column='hour', value=df_train['pickup_datetime'].dt.hour)

bins = [0, 4, 11, 16, 23]

labels = ["12 am to 4 am", "5 am to 11 am", "12 pm to 4 pm", "5 pm to 11 pm"]

df_train.insert(loc=len(df_train.columns) - 1, column='time_category', value=pd.cut(df_train['hour'], bins=bins, labels=labels, include_lowest=True))


In [ ]:
mapping = {
    "12 am to 4 am": 1,
    "5 am to 11 am": 2,
    "12 pm to 4 pm": 4,
    "5 pm to 11 pm": 3
}

df_train['time_category'] = df_train['time_category'].map(mapping)


In [ ]:
df_train.head()

In [ ]:
df_train.drop(columns=[ 'id', 'pickup_datetime', 'store_and_fwd_flag' ,'hour'], inplace=True)

Now We Can Drop Id Column, However we shouldn't dropping the vendor id column because different vendors may providing 
different cars which mabye vendor 1 cars is faster than vendor 2 cars

Now After Dropping Unimportant Columns And Replace Some Of Them by New Columns Like Time Category

The Ascending Order Have Meaning The number 4 Means mostly crowded time Afficing the Time Duration

and 1 is the minmum Crowdness in Roads from my common sense

store_and_fwd_flag, i think this column doesn't have impact on the trip duration so i dropped it 

we Have replaced the pickup datetime

In [ ]:
df_train.head()